# FractalTrader - Multi-Timeframe SMC Viewer

Interactive dashboard for analyzing Smart Money Concepts across multiple timeframes.

**Sprint 1 MVP** - Static historical data analysis

---

## Features

- ✅ **Synchronized 3-panel view** (H4 / H1 / M15)
- ✅ **Auto-detected Order Blocks**
- ✅ **Interactive zoom/pan** (shared time axis)
- 🚧 **Confidence breakdown** (coming next)

---

In [ ]:
# Setup
import sys
sys.path.append('..')  # Add parent directory to path

from visualization import FractalDashboard
import pandas as pd
import numpy as np

print("✅ FractalTrader visualization loaded")

## 1. Generate Sample BTC Data

For Sprint 1 MVP, we'll generate realistic sample data.

**Sprint 2** will fetch live market data from Hyperliquid/Binance.

In [ ]:
# Generate 90 days of 15-minute BTC data
def generate_sample_btc_data(days=90):
    """Generate realistic BTC price movements."""
    
    # 15-minute candles for 90 days
    n_candles = days * 24 * 4
    dates = pd.date_range(
        end=pd.Timestamp.now(),
        periods=n_candles,
        freq='15min'
    )
    
    # Simulate price movement with trend + noise
    np.random.seed(42)
    
    # Base trend
    trend = np.linspace(45000, 50000, n_candles)
    
    # Add realistic volatility
    returns = np.random.randn(n_candles) * 150  # ~$150 std per 15m
    close_prices = trend + np.cumsum(returns)
    
    # Generate OHLC
    df = pd.DataFrame({
        'timestamp': dates,
        'open': close_prices + np.random.randn(n_candles) * 30,
        'high': close_prices + np.abs(np.random.randn(n_candles) * 80),
        'low': close_prices - np.abs(np.random.randn(n_candles) * 80),
        'close': close_prices,
        'volume': np.random.randint(50, 500, n_candles)
    })
    
    return df

# Generate and save
btc_data = generate_sample_btc_data()
btc_data.to_csv('../data/samples/btc_90d.csv', index=False)

print(f"✅ Generated {len(btc_data):,} candles (90 days of BTC data)")
print(f"   Price range: ${btc_data['close'].min():,.0f} - ${btc_data['close'].max():,.0f}")

## 2. Initialize Dashboard

Create a FractalDashboard instance with 3 timeframes.

In [ ]:
dashboard = FractalDashboard(
    pair='BTC/USDT',
    timeframes=['4h', '1h', '15m'],
    min_impulse_percent=0.015  # 1.5% minimum impulse for OB detection
)

print("✅ Dashboard initialized")
print(f"   Pair: {dashboard.pair}")
print(f"   Timeframes: {dashboard.timeframes}")

## 3. Load Data

Load historical data and resample to different timeframes.

In [ ]:
dashboard.load_data('../data/samples/btc_90d.csv')

print("✅ Data loaded and resampled")
for tf, df in dashboard.data.items():
    print(f"   {tf}: {len(df):,} candles")

## 4. Detect SMC Patterns

Run order block detection on all timeframes.

In [ ]:
dashboard.detect_patterns()

print("✅ Order blocks detected")
for tf, (bullish_ob, bearish_ob) in dashboard.order_blocks.items():
    total_obs = len(bullish_ob) + len(bearish_ob)
    valid_obs = (~bullish_ob['invalidated']).sum() + (~bearish_ob['invalidated']).sum()
    print(f"   {tf}: {total_obs} OBs ({valid_obs} valid, {total_obs - valid_obs} invalidated)")

## 5. Display Interactive Dashboard

**Instructions:**
- Zoom: Click and drag on any panel
- Pan: Hold Shift + drag
- Reset: Double-click on chart
- Hover: See OHLC values

**What you're seeing:**
- 🟢 **Green zones** = Bullish Order Blocks (potential buy zones)
- 🔴 **Red zones** = Bearish Order Blocks (potential sell zones)
- **Labels** show retest count (higher = stronger OB)
- **Faded blocks** = Invalidated (price broke through)

In [ ]:
dashboard.show(
    height=900,
    show_invalidated=True,  # Show both valid and invalidated OBs
    max_order_blocks=30     # Limit to top 30 OBs per timeframe
)

## 6. Analyze Specific Order Block

Examine the strongest order blocks on each timeframe.

In [ ]:
# Find strongest bullish OB on 4H (most retests)
bullish_4h, bearish_4h = dashboard.order_blocks['4h']

if len(bullish_4h) > 0:
    strongest_ob = bullish_4h.nlargest(1, 'retest_count').iloc[0]
    
    print("🟢 Strongest Bullish Order Block (4H):")
    print(f"   Time: {strongest_ob.name}")
    print(f"   Zone: ${strongest_ob['ob_low']:,.0f} - ${strongest_ob['ob_high']:,.0f}")
    print(f"   Retests: {strongest_ob['retest_count']}")
    print(f"   Status: {'✓ Valid' if not strongest_ob['invalidated'] else '✗ Invalidated'}")
    print(f"\n   💡 Interpretation: Price has retested this accumulation zone {strongest_ob['retest_count']} times.")
    print(f"      If valid, this is a HIGH PROBABILITY support level.")
else:
    print("No bullish order blocks detected on 4H")

## 7. Export Chart (Optional)

Save as HTML for sharing or later review.

In [ ]:
fig = dashboard.render(height=1000)
fig.write_html('../data/samples/btc_fractal_analysis.html')

print("✅ Chart saved to data/samples/btc_fractal_analysis.html")
print("   Open in browser to view offline")

---

## 🚀 Next Steps (Sprint 2)

- [ ] Live data integration (Hyperliquid/Binance)
- [ ] Real-time updates (auto-refresh)
- [ ] FVG (Fair Value Gap) overlays
- [ ] Liquidity sweep markers
- [ ] Confidence scoring panel
- [ ] Trade journal component

---

## 📚 Learn More

- **Order Blocks**: Last opposite candle before impulse (see `core/order_blocks.py`)
- **SMC Theory**: [docs/archive/fractal-trader-context.md](../docs/archive/fractal-trader-context.md)
- **Sprint Plan**: [docs/ROADMAP_Q1_2025.md](../docs/ROADMAP_Q1_2025.md)

---

**FractalTrader** - Open source SMC trading system

🚢 Sprint 1 Deliverable (Dec 24 - Jan 6, 2025)